In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
sys.path.append("..")

In [64]:
import time
import json
import numpy as np

In [62]:
from bdds.node_bdds import MapsInfo
from rom_tools.rom_manager import RomManager
from abstraction_validation.abstractify import abstractify_pos_global
from world_rando.coord import Coord

In [38]:
rom = RomManager("../../roms/sm_clean.sfc", "sm_junk2.smc")

In [39]:
map_info = MapsInfo("../encoding/dsl/rooms_for_alloy.txt", \
                   "../encoding/dsl/exits_for_alloy.txt",
                   "../output/ram_snes9x.bin",
                   rom)

Landing_Site 0x791f8
	 Extra: []
	 Missing: ['Landing_Site_Ship']
West_Ocean 0x793fe
	 Extra: ['West_Ocean_R4', 'West_Ocean_L']
	 Missing: ['West_Ocean_L2', 'West_Ocean_R6']
Bomb_Torizo 0x79804
	 Extra: ['Bomb_Torizo_B']
	 Missing: ['Bomb_Torizo_Bombs']
Spore_Spawn 0x79dc7
	 Extra: []
	 Missing: ['Spore_Spawn_Spore_Spawn']
Kraid 0x7a59f
	 Extra: []
	 Missing: ['Kraid_Kraid']
Statues 0x7a66a
	 Extra: []
	 Missing: ['Statues_Statues']
Crocomire 0x7a98d
	 Extra: []
	 Missing: ['Crocomire_Crocomire']
Water_Closet 0x7b1e5
	 Extra: []
	 Missing: ['Water_Closet_Drain']
Golden_Torizo 0x7b283
	 Extra: []
	 Missing: ['Golden_Torizo_Golden_Torizo']
Ridley 0x7b32e
	 Extra: []
	 Missing: ['Ridley_Ridley']
Phantoon 0x7cd13
	 Extra: []
	 Missing: ['Phantoon_Phantoon']
West_Sand_Hall 0x7d461
	 Extra: []
	 Missing: ['West_Sand_Hall_TS']
East_Sand_Hall 0x7d4c2
	 Extra: []
	 Missing: ['East_Sand_Hall_TS']
West_Sand_Hole 0x7d4ef
	 Extra: []
	 Missing: ['West_Sand_Hole_TS']
East_Sand_Hole 0x7d51e
	 Extra: 

In [49]:
posd = {
    "X": None,
    "Y": None
}

In [63]:
# The map on the website from bin0al uses a slightly different coordinate system
map_area_offsets = {
0: Coord(3, 10),
1: Coord(0, 28),
2: Coord(31, 48),
3: Coord(37, 0),
4: Coord(28, 28),
5: Coord(0, 10),
#Out of bounds
6: Coord(0, -10),
7: Coord(0, 0),
}

In [65]:
def get_onmap_pos():
    ram_data = np.fromfile("../output/ram_snes9x.bin", dtype="int16")
    return abstractify_pos_global(ram_data, map_area_offsets)

In [68]:
from data_types.constraintgraph import Item, Boss

In [74]:
item_nodes = set([])
for r, room in map_info.rooms.items():
    for node in room.graph.name_node.keys():
        n = room.graph.name_node[node].data
        if isinstance(n, Item) or isinstance(n, Boss):
            item_nodes.add(node)

In [75]:
from itertools import pairwise

In [4]:
# Yellow: current segment
# Green: segment up to the next item
# Grey: rest of the lines
def generate_lines(state_estimate, policy):
    path = map_info.get_path_advice(state_estimate, policy)
    if path is None or len(path) == 0:
        return None
    else:
        # Filter down to nodes that we know the location of
        #TODO: this has issues if there's an item node with an unknown location (e.g. a boss)
        path_locs = [(p, map_info.all_posns[p]) for p in path if p in map_info.all_posns]
        out = {
            "next_step": [],
            "next_item": [],
            "remaining_path": [],
        }
        current = state_estimate[0]
        posns = []
        if current in map_info.all_posns:
            path_locs.insert((current, map_info.all_posns[current]), 0)
        which = "next_step"
        for ((node_a, loc_a), (node_b, loc_b)) in pairwise(path_locs):
            out[which].append((loc_a, loc_b))
            if which == next_step:
                which = "next_item"
            if which == next_item and node_b in item_nodes:
                which = "remaining_path"
        return out

In [5]:
while True:
    #pos = map_info.read_state().position
    pos = get_onmap_pos()
    posd["X"] = int(pos.x)
    posd["Y"] = int(pos.y)
    dstr = json.dumps(posd)
    with open("player_pos.json", "w") as f:
        f.write(dstr)
    time.sleep(0.1)

NameError: name 'get_onmap_pos' is not defined

In [66]:
print(posd)
print(get_onmap_pos())

{'X': 468, 'Y': 629}
c(468,613)
